In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import copy
from fake_headers import Headers
import time
import numpy as np
import os

### Introduction

In [2]:
def write_file(dictionary,name):
    with open(name, 'w', encoding='utf-8') as outfile:
        json.dump(dictionary, outfile, ensure_ascii=False)
        
def read_file(file):        
    with open(file, encoding='utf-8') as json_file:
        data = json.load(json_file)
    
    return data

## Paso 1
Recolectar nombres de provincias

In [3]:
def get_provincias():
    res = requests.get('https://cornershopapp.com/es-pe/')
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text,'lxml')
    list_html_op = soup.find(id='address').find_all('option')[1:]
    list_distrito = []
    for i in list_html_op:
        so = BeautifulSoup(i.encode(),'lxml')
        list_distrito.append(so.body.option.attrs['value'])
    return list_distrito

In [4]:
list_dis = get_provincias()
list_dis

['Ancon',
 'Asia',
 'Ate',
 'Barranco',
 'Bellavista',
 'Breña',
 'Callao',
 'Carabayllo',
 'Carmen De La Legua Reynoso',
 'Chaclacayo',
 'Chorrillos',
 'Cieneguilla',
 'Comas',
 'El Agustino',
 'Independencia',
 'Jesus Maria',
 'La Molina',
 'La Perla',
 'La Punta',
 'La Victoria',
 'Lima',
 'Lince',
 'Los Olivos',
 'Lurigancho',
 'Lurin',
 'Magdalena Del Mar',
 'Magdalena Vieja',
 'Mi Perãz',
 'Miraflores',
 'Pachacamac',
 'Pucusana',
 'Pueblo Libre',
 'Puente Piedra',
 'Punta Hermosa',
 'Punta Negra',
 'Rimac',
 'San Bartolo',
 'San Borja',
 'San Isidro',
 'San Juan De Lurigancho',
 'San Juan De Miraflores',
 'San Luis',
 'San Martin De Porres',
 'San Miguel',
 'Santa Anita',
 'Santa Maria Del Mar',
 'Santa Rosa',
 'Santiago De Surco',
 'Surquillo',
 'Ventanilla',
 'Villa El Salvador',
 'Villa Maria Del Triunfo']

### Paso 2
Por provincia get categorias de tiendas y tiendas
	- url - payload : locality, country
	- url : https://cornershopapp.com/api/v2/stores?locality=Jesus+Maria&country=PE
	- json recibido>por categoria -> {cat,cat,cat}
	- categoria(Featured = top) y stores [{name:'',stores:''}]
	- stores:[{id:'',name:'',self_picking,self_delivery,*closest_branch*}]
	-*closest_branch {id,lat,lng,name,store_name,address,city,has_same_prices, delivery_slots=[]}


In [5]:
def market_by_district(district):
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'es-PE;q=0.8,es;q=0.7,en-US;q=0.6,en;q=0.5,es-US;q=0.4,ko-KR;q=0.3,ko;q=0.2',
    'Cache-Control': 'no-cache'}
    #https://web.cornershopapp.com/?locality=Jesus%20Maria&country=PE
    payload = {'locality':district,'country':'PE'}
    hdrs = Headers(headers=True).generate()
    hdrs['Accept-Language'] = 'es-PE;q=0.8,es;q=0.7'
    r = requests.get('https://cornershopapp.com/api/v2/stores',params=payload, headers=hdrs)
    if r.status_code != 200:
        print('Error market_by_district ',r.status_code)
    print(r.url)
    json_res = r.json()
    
    return json_res

In [6]:
#Create folders
true_dis = []
false_dis = []
for i in list_dis:
    json_markets = market_by_district(i)
    if json_markets == []:
        false_dis.append(i)
    else:
        true_dis.append(i)
        i = i.replace(" ","")
        path = 'D:/PyScripts/ws_anthony/cornershop/'+i.lower()
        os.mkdir(path)

true_dis

https://cornershopapp.com/api/v2/stores?locality=Ancon&country=PE
https://cornershopapp.com/api/v2/stores?locality=Asia&country=PE
https://cornershopapp.com/api/v2/stores?locality=Ate&country=PE
https://cornershopapp.com/api/v2/stores?locality=Barranco&country=PE


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:/PyScripts/ws_anthony/cornershop/barranco'

In [6]:
#json_markets[1]['stores'][0]['closest_branch']

In [7]:
json_markets = market_by_district('Miraflores')
json_markets

https://cornershopapp.com/api/v2/stores?locality=Miraflores&country=PE


[{'id': 'featured',
  'name': 'Destacados',
  'style': {'type': 'classic', 'attributes': {}},
  'stores': [{'id': 872,
    'name': 'Vivanda',
    'country': 'PE',
    'locale': 'es_PE',
    'priority': 10,
    'img_url': 'https://s.cornershopapp.com/store-logo/baecf757-ba2.png?versionId=Kr.FjURKmiR.PzMyefR7ezP9LFeSSTx9',
    'light_img_url': 'https://s.cornershopapp.com/store-logo/a58c48dd-440.png?versionId=SGm4t37Rlt.Zp.V.US4bt3KbYOTohf2J',
    'self_picking': False,
    'self_delivery': False,
    'minimum_amount': None,
    'maximum_amount': None,
    'closest_branch': {'id': 2979,
     'lat': -12.124493,
     'lng': -77.028046,
     'picking_point': {'type': 'Point',
      'coordinates': [-77.028046, -12.124493]},
     'name': 'Benavides',
     'description': 'Un concepto original y único, dirigido a convertir la compra diaria de nuestros clientes en una experiencia gratificante.',
     'description_disclaimer': None,
     'store_id': 872,
     'store_name': 'Vivanda',
     'store_

#### Nota:
Analizar atributo de branch - > delivery slots

In [8]:
def format_market_by_category(data):
    list_st = data['stores'] #list of all stores in one category
    name_category_market = data['name']
    for i in list_st:
        i['id_branch'] = i['closest_branch'].get('id') 
        i['lat_branch'] = i['closest_branch'].get('lat')
        i['lng_branch'] = i['closest_branch'].get('lng')
        i['name_branch'] = i['closest_branch'].get('name')
        i['addr_branch'] = i['closest_branch'].get('address')
        i['city_branch'] = i['closest_branch'].get('city')
        i['same_prices_branch'] = i['closest_branch'].get('has_same_prices')
        
    df = pd.DataFrame(list_st)
    df['cat_market'] = name_category_market
    df = df[['cat_market','id','name','country','priority','self_picking','self_delivery','id_branch','lat_branch',
             'lng_branch','name_branch','addr_branch','city_branch','same_prices_branch']]
    return df
        

In [9]:
top_markets = copy.deepcopy(json_markets)[0] #dict
markets = copy.deepcopy(json_markets)[1:] #list

In [10]:
if markets == []:
    markets = copy.deepcopy(json_markets)

In [11]:
# get top markets in list with id
list_top_market = format_market_by_category(top_markets)['id'].tolist()
list_top_market

[872, 842, 892, 1475, 1278, 1957, 1225, 1274]

In [12]:
markets

[{'id': 'new_stores',
  'name': 'Nuevas tiendas',
  'style': {'type': 'posters', 'attributes': {}},
  'stores': [{'id': 2954,
    'name': 'Babycottons',
    'country': 'PE',
    'locale': 'es_PE',
    'priority': None,
    'img_url': 'https://s.cornershopapp.com/store-logo/d74927b2-7eb.png?versionId=jhsM5Vybavcj9A3CfB.byHHW6OHGlF8Q',
    'light_img_url': 'https://s.cornershopapp.com/store-logo/6876f06c-f5c.png?versionId=S0XiP873DF3MF2NFe9GLlHwXjLYm1Nyb',
    'self_picking': False,
    'self_delivery': False,
    'minimum_amount': 0.0,
    'maximum_amount': None,
    'closest_branch': {'id': 8836,
     'lat': -12.099249,
     'lng': -77.036231,
     'picking_point': {'type': 'Point',
      'coordinates': [-77.036231, -12.099249]},
     'name': 'Conquistadores - San Isidro',
     'description': 'Ofrecemos ropa y accesorios para bebés y niños hasta los 6 años.',
     'description_disclaimer': None,
     'store_id': 2954,
     'store_name': 'Babycottons',
     'store_img_url': 'https://s.c

In [13]:
# iter each market and format it
def process_markets_by_category(all_markets):
    df_all = pd.DataFrame([],columns=[])
    
    markets_filtered = [] # remove markets categorized in new_stores
    for market in all_markets:
        if market['id'] != 'new_stores':
            markets_filtered.append(market)
    
    for market in markets_filtered:
        df_temp = format_market_by_category(market)
        df_all = df_all.append(df_temp)
    return df_all

In [14]:
df_markets_by_cat = process_markets_by_category(markets)
df_markets_by_cat['id_branch'].apply(lambda x: str(x)).describe()

count       27
unique      27
top       4867
freq         1
Name: id_branch, dtype: object

In [15]:
df_markets_by_cat

,cat_market,id,name,country,priority,self_picking,self_delivery,id_branch,lat_branch,lng_branch,name_branch,addr_branch,city_branch,same_prices_branch
0,Panadería y pastelería,1274,Dolce Capriccio,PE,None,False,False,3804,-12.117826,-77.035594,Miraflores,Calle Dos de Mayo 694 Miraflores,LIM,True
1,Panadería y pastelería,1187,CheesecakeToGo.PE,PE,None,False,False,4343,-12.119838,-77.034471,Edgard Castro,"Av Grau 171 Miraflores, Lima, Lima",LIM,True
2,Panadería y pastelería,1353,La p'tite France,PE,None,False,False,4105,-12.116545,-77.024003,Surquillo,"Jr. Gonzales Prada 599, Surquillo",LIM,True
3,Panadería y pastelería,1177,Limapostre,PE,None,False,False,3571,-12.121107,-76.996293,Surco,"Calle Maria Reiche 167, Surco",LIM,True
4,Panadería y pastelería,1341,La Nuez,PE,None,False,False,4042,-12.128152,-76.992398,Benavides,"av. benavides 4121 surco, La Nuez - Benavides",LIM,True
5,Panadería y pastelería,1183,San Jorge,PE,None,False,False,3608,-12.088449,-77.015761,Av. Carlos Villarán,"Av. Carlos Villarán 1050, La Victoria, Lima, Perú",LIM,True
0,Vida sana,1957,Más Natural House,PE,None,False,False,5769,-12.104603,-77.000972,Avenida Aviación,Avenida Aviacion 3367,LIM,True
1,Vida sana,1791,Al Gramo,PE,None,False,False,5415,-12.104561,-77.000950,San Borja,"avenida aviacion 3367, San Borja",LIM,True
2,Vida sana,1663,Thika Thani,PE,None,False,False,4743,-12.124824,-77.028301,Miraflores,Av. Alfredo Benavides 474 Miraflores,LIM,True
0,Mascotas,1475,Groomers,PE,6,False,False,4397,-12.125914,-77.032318,28 de julio,"Av 28 de Julio 200, Miraflores",LIM,True


In [16]:
#add label of top_market and rename columns and normalizar cat_market
def set_top_markets(df_markets,ls_top):
    df_markets['top_market'] = df_markets['id'].apply(lambda x: 1 if x in ls_top else 0)
    df_markets = df_markets.rename(columns={'id': 'id_market','name':'name_market'})
    # 1.
    #df_markets['n_catm'] = pd.factorize(dfall['cat_market'])[0]
    #dfall['ncat'] = pd.factorize(dfall['categoria'])[0]
    # 2
    #dfall['cat_sub'] = dfall['ncat'].map(str) + '_' + dfall['nsub'].map(str)
    return df_markets

In [17]:
#markets with label of top_market
df_markets_featured = set_top_markets(df_markets_by_cat, list_top_market)
#df_markets_featured

In [18]:
df_markets_featured[['name_market','cat_market']].describe()

,name_market,cat_market
count,27,27
unique,27,9
top,CheesecakeToGo.PE,Panadería y pastelería
freq,1,6


In [19]:
df_markets_featured.shape

(27, 15)

In [20]:
def build_tuple_ex(y):
    return (y['id_branch'],y['id_market'])

In [21]:
#list_id_branch = df_markets_featured[['id_branch','name_market']].tolist()
#list_id_branch
df_markets_featured['extra'] = df_markets_featured.apply(build_tuple_ex,axis=1)
lista_ex = df_markets_featured['extra'].tolist()
len(lista_ex)

27

###### export

In [22]:
df_markets_featured

,cat_market,id_market,name_market,country,priority,self_picking,self_delivery,id_branch,lat_branch,lng_branch,name_branch,addr_branch,city_branch,same_prices_branch,top_market,extra
0,Panadería y pastelería,1274,Dolce Capriccio,PE,None,False,False,3804,-12.117826,-77.035594,Miraflores,Calle Dos de Mayo 694 Miraflores,LIM,True,1,"(3804, 1274)"
1,Panadería y pastelería,1187,CheesecakeToGo.PE,PE,None,False,False,4343,-12.119838,-77.034471,Edgard Castro,"Av Grau 171 Miraflores, Lima, Lima",LIM,True,0,"(4343, 1187)"
2,Panadería y pastelería,1353,La p'tite France,PE,None,False,False,4105,-12.116545,-77.024003,Surquillo,"Jr. Gonzales Prada 599, Surquillo",LIM,True,0,"(4105, 1353)"
3,Panadería y pastelería,1177,Limapostre,PE,None,False,False,3571,-12.121107,-76.996293,Surco,"Calle Maria Reiche 167, Surco",LIM,True,0,"(3571, 1177)"
4,Panadería y pastelería,1341,La Nuez,PE,None,False,False,4042,-12.128152,-76.992398,Benavides,"av. benavides 4121 surco, La Nuez - Benavides",LIM,True,0,"(4042, 1341)"
5,Panadería y pastelería,1183,San Jorge,PE,None,False,False,3608,-12.088449,-77.015761,Av. Carlos Villarán,"Av. Carlos Villarán 1050, La Victoria, Lima, Perú",LIM,True,0,"(3608, 1183)"
0,Vida sana,1957,Más Natural House,PE,None,False,False,5769,-12.104603,-77.000972,Avenida Aviación,Avenida Aviacion 3367,LIM,True,1,"(5769, 1957)"
1,Vida sana,1791,Al Gramo,PE,None,False,False,5415,-12.104561,-77.000950,San Borja,"avenida aviacion 3367, San Borja",LIM,True,0,"(5415, 1791)"
2,Vida sana,1663,Thika Thani,PE,None,False,False,4743,-12.124824,-77.028301,Miraflores,Av. Alfredo Benavides 474 Miraflores,LIM,True,0,"(4743, 1663)"
0,Mascotas,1475,Groomers,PE,6,False,False,4397,-12.125914,-77.032318,28 de julio,"Av 28 de Julio 200, Miraflores",LIM,True,1,"(4397, 1475)"


In [23]:
df_markets_featured['id_branch'].apply(lambda x: str(x)).describe()

count       27
unique      27
top       4867
freq         1
Name: id_branch, dtype: object

### Paso 3
Get departamentos por branches(tienda mas cercana)
	- url-payload : id_branch,locality,country
	- url:https://cornershopapp.com/api/v1/branches/3071?with_suspended_slots&locality=Jesus+Maria&country=PE
	- json recibido -> {branch,departments,recent_products}
	- por departamento : {id,name,top_products,aisles}
	- guardar id_departamento
    
   ### Paso 4
     Get aisles(subcategorias)
        - url-payload : id_branch,id_departamento
        - url:https://cornershopapp.com/api/v1/branches/2896/departments/C_538
        - json recibido -> {id,name,img_url,top_products,aisles}
        - (subcat)guardar por cada aisles : {id,name,num_products,top_products}

### FALTA: Iterar todos los branches segun market y segun categoria
idea: crear un id : normalizar(cat_market)_(id_market)_(id_branch)

#### Test with one branch

In [24]:
# 2902 - PlazaVea
# gather branch data and departments id
def departments_by_branch(id_branch, district):
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'es-PE;q=0.8,es;q=0.7,en-US;q=0.6,en;q=0.5,es-US;q=0.4,ko-KR;q=0.3,ko;q=0.2',
    'Cache-Control': 'no-cache'}
    #https://cornershopapp.com/api/v1/branches/3071?with_suspended_slots&locality=Jesus+Maria&country=PE
    payload = {'locality':district,'country':'PE'}
    hdrs = Headers(headers=True).generate()
    hdrs['Accept-Language'] = 'es-PE;q=0.8,es;q=0.7'
    r = requests.get('https://cornershopapp.com/api/v1/branches/'+id_branch+'?with_suspended_slots',params=payload, headers=hdrs)
    if r.status_code != 200:
        print('Error departments_by_branch ',r.status_code)
    print(r.url)
    json_res = r.json()
    
    return json_res

In [69]:
json_branch = departments_by_branch('3804','Barranco')

https://cornershopapp.com/api/v1/branches/3804?with_suspended_slots&locality=Barranco&country=PE


In [70]:
json_depas = json_branch['departments']
#json_depas

In [25]:
# branch - market : 2902 - PlazaVea
# get all aisles
def subcat_by_depa(id_branch, id_depa):
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'es-PE;q=0.8,es;q=0.7,en-US;q=0.6,en;q=0.5,es-US;q=0.4,ko-KR;q=0.3,ko;q=0.2',
    'Cache-Control': 'no-cache'}
    #https://cornershopapp.com/api/v1/branches/2896/departments/C_538
    payload = {}
    hdrs = Headers(headers=True).generate()
    hdrs['Accept-Language'] = 'es-PE;q=0.8,es;q=0.7'
    r = requests.get('https://cornershopapp.com/api/v1/branches/'+id_branch+'/departments/'+id_depa,params=payload, headers=hdrs)
    if r.status_code != 200:
        print('Error subcat_by_depa ',r.status_code)
    print(r.url)
    json_res = r.json()
    
    return json_res

In [26]:
def format_subcategories_by_deparment(data,id_branch, cont_aux):
    list_depas = data['departments'] #list of all stores in one category
    
    list_by_subcat = []
    temp_dict = {}
    for depa in list_depas:
        id_depa = depa['id']
        name_depa = depa['name']
        
        # requests
        url = 'https://cornershopapp.com/api/v1/branches/'+id_branch+'/departments/'+id_depa
        cont_aux += 1
        json_subcat = do_requests({},url,cont_aux)
        #json_subcat = subcat_by_depa(id_branch,id_depa)
        
        #list_subcats = depa['aisles']
        list_subcats = json_subcat['aisles']
        
        for subcat in list_subcats:
            id_subcat = subcat['id']
            name_subcat = subcat['name']
            num_products = subcat['num_products']
            top_products = [pro['id'] for pro in subcat['top_products']] #lista
            
            temp_dict['id_depa'] = id_depa
            temp_dict['name_depa'] = name_depa
            temp_dict['id_subcat'] = id_subcat
            temp_dict['name_subcat'] = name_subcat
            temp_dict['num_products'] = num_products
            temp_dict['top_products'] = top_products
            
            list_by_subcat.append(temp_dict)
            temp_dict = {}
        
        if not list_subcats:
            temp_dict['id_depa'] = id_depa
            temp_dict['name_depa'] = name_depa
            temp_dict['id_subcat'] = ''
            temp_dict['name_subcat'] = ''
            temp_dict['num_products'] = 0
            temp_dict['top_products'] = []
            
            list_by_subcat.append(temp_dict)
            temp_dict = {}
        
    df = pd.DataFrame(list_by_subcat)
    #print(df.head())
    return df, cont_aux

In [95]:
#3804/departments/C_509
df_subcat,idx = format_subcategories_by_deparment(json_branch,'3804',1)
df_subcat

https://cornershopapp.com/api/v1/branches/3804/departments/C_45
https://cornershopapp.com/api/v1/branches/3804/departments/C_509


,id_depa,name_depa,id_subcat,name_subcat,num_products,top_products
0,C_45,Pastelería,,,0,[]
1,C_509,Comidas preparadas,,,0,[]


In [97]:
df_subcat[['id_depa','name_depa','id_subcat','name_subcat']].describe()

,id_depa,name_depa,id_subcat,name_subcat
count,2,2,2,2
unique,2,2,1,1
top,C_509,Comidas preparadas,,
freq,1,1,2,2


In [27]:
def build_tuple_aux(y):
    return (y['id_depa'],y['id_subcat'])

In [99]:
#df_subcat['id_depa'].unique()
df_subcat['extra'] = df_subcat.apply(build_tuple_aux,axis=1)
df_subcat['extra'].tolist()

[('C_45', ''), ('C_509', '')]

In [100]:
df_subcat.insert(0,"id_branch","3804")

###### export

In [101]:
df_subcat.head()

,id_branch,id_depa,name_depa,id_subcat,name_subcat,num_products,top_products,extra
0,3804,C_45,Pastelería,,,0,[],"(C_45, )"
1,3804,C_509,Comidas preparadas,,,0,[],"(C_509, )"


### FALTA: Iterar todos los subcat segun depa
idea: crear un id : normalizar(id_dep)_(id_subcat)

### Paso 5
Get all products por categoria
	- payload : id_branch,id_aisle
	- url:https://cornershopapp.com/api/v1/branches/2896/aisles/C_134/products
	- json_recibido -> [{producto},{producto},{producto},etc]
	- * el json_recibido esta ordenado (los 10 primeros son prod. top de aisles)
	- por producto {id,brand,buy_unit,currency,price,pricing{price_per_unit{unit,amount,currency}},nutritional_info,name,description,img_url}

In [28]:
# 2902 - PlazaVea
# gather branch data and departments id
def products_by_aisle(id_branch, id_aisle):
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'es-PE;q=0.8,es;q=0.7,en-US;q=0.6,en;q=0.5,es-US;q=0.4,ko-KR;q=0.3,ko;q=0.2',
    'Cache-Control': 'no-cache'}
    #https://cornershopapp.com/api/v1/branches/2896/aisles/C_134/products
    payload = {}
    hdrs = Headers(headers=True).generate()
    hdrs['Accept-Language'] = 'es-PE;q=0.8,es;q=0.7'
    r = requests.get('https://cornershopapp.com/api/v1/branches/'+id_branch+'/aisles/'+id_aisle+'/products',params=payload, headers=hdrs)
    if r.status_code != 200:
        print('Error products_by_aisle ',r.status_code)
    print(r.url)
    json_res = r.json()
    
    return json_res

In [128]:
#https://cornershopapp.com/api/v1/branches/7899/aisles/C_134/products
list_products = products_by_aisle('7899','C_134')
#list_products = [{"id":1849648,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":6.69,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":6.69,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Palta Fuerte","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849648.jpg?versionId=h6XZdO1SAuS3xBK8OrYXQnp86Gh5oGlV"},{"id":1849605,"brand":"","buy_unit":"UN","default_buy_unit":"UN","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":2.29,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":2.29,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Platano De Seda Bolsa","description":"","package":"Paquete De 6 Unidades","img_url":"https://s.cornershopapp.com/product-images/1849605.jpg?versionId=VUfFoRBVm5DAiO.Y6mvQd6EefTrB16aJ"},{"id":1849532,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":6.25,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":6.25,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Tomate Italiano","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849532.jpg?versionId=6HZc6Ioi7kH0HUffxWScDIyctHKgAenk"},{"id":1849949,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":4.79,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":4.79,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Limon Sutil","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849949.jpg?versionId=ZkZlIGwAxuO75T1_gIEdVtryegmd71dg"},{"id":1849800,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":5.99,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":5.99,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Aji Verde","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849800.jpg?versionId=VhSSv0U_fan.pSBmurvfB4OkgXs9ff5o"},{"id":1849804,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":5.99,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":5.99,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Pimiento Morron","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849804.jpg?versionId=ejkmWRifIczTcRJmTpb_pDNS_ie124RW"},{"id":1849659,"brand":"","buy_unit":"UN","default_buy_unit":"UN","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":6.99,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":6.99,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Maracuya ","description":"","package":"Malla De 2 Kg.","img_url":"https://s.cornershopapp.com/product-images/1849659.jpg?versionId=XEETE0T4i0A6xQ5RrsesJPsqiSpvASoV"},{"id":1849608,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":4.79,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":4.79,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Papaya","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849608.jpg?versionId=godPWxdUNDJK4jqRTZtw9f3.iY6ZRaet"},{"id":1849945,"brand":"","buy_unit":"UN","default_buy_unit":"UN","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":15.95,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":15.95,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Naranja Jugo Especial ","description":"","package":"Malla De 5 Kg.","img_url":"https://s.cornershopapp.com/product-images/1849945.jpg?versionId=i2XsqmXEtQcpHRVz9bQGCyJtQAereyrT"},{"id":1849604,"brand":"","buy_unit":"UN","default_buy_unit":"UN","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":4.79,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":4.79,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Platano Bizcocho En Bandeja","description":"","package":"Unidad","img_url":"https://s.cornershopapp.com/product-images/1849604.jpg?versionId=SWG_NvWgEuEa5GJMHsOIARlaJ79a928y"},{"id":1849600,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":5.99,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":5.99,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Pina Golden Premium","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849600.jpg?versionId=aP0SIQ5ZnPJqb0js20Y_Er5M.X5_xLeJ"},{"id":1849615,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":2.29,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":2.29,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Mandarina Satsuma","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849615.jpg?versionId=PY9JoYgXS042oODuE2t8tNd3RZbGn98i"},{"id":1849909,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":2.49,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":2.49,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Mango Kent","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849909.jpg?versionId=OLoqTZMreC_RVSb3mFQdcRKWo6tV7mMp"},{"id":1849629,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":8.99,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":8.99,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Fresa Entera Bandeja","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849629.jpg?versionId=ABZyodi4BswOvxgeaVY0BvE46kDZczbl"},{"id":1849805,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":4.19,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":4.19,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Zapallo Macre","description":"","package":"1Kg.","img_url":"https://s.cornershopapp.com/product-images/1849805.jpg?versionId=qPzFwXxkjHqRw1Fvv1aasiMumzegKdkU"},{"id":1849607,"brand":"","buy_unit":"UN","default_buy_unit":"UN","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":31.9,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":31.9,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Papaya Light","description":"","package":"Caja De 10 Kg.","img_url":"https://s.cornershopapp.com/product-images/1849607.jpg?versionId=x4z3mvKrIivqIq6Mp9hA2NPr3IUz4ucl"},{"id":1849475,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":2.49,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":2.49,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"MelÃ³n Coquito","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849475.jpg?versionId=0p56sHx1ocoR4NHIuCJ92T.9i65SaSeu"},{"id":1849905,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":3.49,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":3.49,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Pina Hawaii","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849905.jpg?versionId=u6XgeBtBdrAlenxBdkChUc.fRtLpkeO0"},{"id":1481509,"brand":{"id":31318,"name":"Ecofresh"},"buy_unit":"UN","default_buy_unit":"UN","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":7.49,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":7.49,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Jugo De Toronja Ecofresh Botella","description":"","package":"Unidad X 500 Ml.","img_url":"https://s.cornershopapp.com/product-images/1481509.jpg?versionId=bkp1iF057VMgOHXVFfBOHE83skJpVude"},{"id":1849474,"brand":"","buy_unit":"KG","default_buy_unit":"KG","variable_weight":"False","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":1.69,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":1.69,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Sandia","description":"","package":"Kilogramo","img_url":"https://s.cornershopapp.com/product-images/1849474.jpg?versionId=_LXRTBvMIxkwmhm.KghbROnHh5ruMK3a"},{"id":1849614,"brand":"","buy_unit":"UN","default_buy_unit":"UN","variable_weight":"","currency":"PEN","unit_conversion_rate":"","purchasable":"True","pricing":{"original_price":"","price":{"label":"Price","amount":44.2,"currency":"PEN"},"price_per_unit":""},"price_per_unit":"","price":44.2,"original_price":"","label":"","description_sections":[],"availability_status":"AVAILABLE","nutritional_info":"","name":"Naranja Morocha","description":"","package":"Malla de 15 Kg","img_url":"https://s.cornershopapp.com/product-images/1849614.jpg?versionId=ad0GxmKcFCvi9jeuNKUz1ScvR1uhDzNd"}]

https://cornershopapp.com/api/v1/branches/7899/aisles/C_134/products


In [29]:
def format_products(ls_products):
    
    process_ls_product = []
    temp_dict_pro = {}
    for product in ls_products:
        id_product = product['id']
        branding = product['brand']
        id_brand = ''
        name_brand = ''
        if branding != None:
            id_brand = branding.get('id','')
            name_brand = branding.get('name','')
        buy_unit = product['buy_unit']
        currency = product['currency']
        unit_conversion_rate = product['unit_conversion_rate']
        purchasable = product['purchasable']
        pricing = product['pricing'].get('price_per_unit','')
        ppu_unit = ''
        ppu_amount = ''
        if pricing != None:
            ppu_unit = pricing['unit']
            ppu_amount = pricing['amount']
        price = product['price']
        availability_status = product['availability_status']
        nutritional_info = product['nutritional_info']
        name = product['name']
        description = product['description']
        package = product['package']
        img_url = product['img_url']

        temp_dict_pro['id_product'] = id_product
        temp_dict_pro['id_brand'] = id_brand
        temp_dict_pro['name_brand'] = name_brand
        temp_dict_pro['buy_unit'] = buy_unit
        temp_dict_pro['currency'] = currency
        temp_dict_pro['unit_conversion_rate'] = unit_conversion_rate
        temp_dict_pro['purchasable'] = purchasable
        temp_dict_pro['ppu_unit'] = ppu_unit
        temp_dict_pro['ppu_amount'] = ppu_amount
        temp_dict_pro['price'] = price
        temp_dict_pro['availability_status'] = availability_status
        temp_dict_pro['nutritional_info'] = nutritional_info
        temp_dict_pro['name'] = name
        temp_dict_pro['description'] = description
        temp_dict_pro['package'] = package
        temp_dict_pro['img_url'] = img_url

        process_ls_product.append(temp_dict_pro)
        temp_dict_pro = {}


    df_product_filtered = pd.DataFrame(process_ls_product)
    df_product_filtered.fillna('', inplace = True)
    
    return df_product_filtered



In [130]:
df_products = format_products(list_products)

In [30]:
#add label of top_product and rename columns
def set_top_products(df_product,ls_top_prod):
    #print(ls_top_prod)
    df_product['top_product'] = df_product['id_product'].apply(lambda x: 1 if x in ls_top_prod else 0)
    #df_markets = df_markets.rename(columns={'id': 'id_market','name':'name_market'})
    return df_product

In [132]:
lista_top_by_subcat = df_subcat['top_products'].tolist()
unique_list_top_product = []
for ls in lista_top_by_subcat:
    unique_list_top_product.extend(ls)

df_products = set_top_products(df_products,unique_list_top_product)
df_products.head()

,id_product,id_brand,name_brand,buy_unit,currency,unit_conversion_rate,purchasable,ppu_unit,ppu_amount,price,availability_status,nutritional_info,name,description,package,img_url,top_product
0,1849648,,,KG,PEN,,True,,,6.69,AVAILABLE,,Palta Fuerte,,Kilogramo,https://s.cornershopapp.com/product-images/184...,0
1,1849605,,,UN,PEN,,True,,,2.29,AVAILABLE,,Platano De Seda Bolsa,,Paquete De 6 Unidades,https://s.cornershopapp.com/product-images/184...,0
2,1849532,,,KG,PEN,,True,,,6.25,AVAILABLE,,Tomate Italiano,,Kilogramo,https://s.cornershopapp.com/product-images/184...,0
3,1849949,,,KG,PEN,,True,,,4.79,AVAILABLE,,Limon Sutil,,Kilogramo,https://s.cornershopapp.com/product-images/184...,0
4,1849800,,,KG,PEN,,True,,,5.99,AVAILABLE,,Aji Verde,,Kilogramo,https://s.cornershopapp.com/product-images/184...,0


In [67]:
name_file = str('7899') + '_' + str('C_134') #branch_aisle
#https://cornershopapp.com/api/v1/branches/7899/aisles/C_134/products
df_products.to_csv('1_'+name_file+'.csv',index=False,header=True,encoding='utf-8')

### Main 

In [57]:
# market_by_district(district)
# payload = {'locality':district,'country':'PE'}
# 'https://cornershopapp.com/api/v2/stores'

# departments_by_branch
# payload = {'locality':district,'country':'PE'}
# https://cornershopapp.com/api/v1/branches/'+id_branch+'?with_suspended_slots

#subcat_by_depa(id_branch, id_depa):
# payload = {}
# 'https://cornershopapp.com/api/v1/branches/'+id_branch+'/departments/'+id_depa

# products_by_aisle(id_branch, id_aisle):
# payload = {}
# https://cornershopapp.com/api/v1/branches/'+id_branch+'/aisles/'+id_aisle+'/products

#ip_addresses = ['187.86.137.141:58918', '87.247.3.234:31444', '162.244.83.133:3128', '201.88.74.34:55443', '168.181.62.74:36928',
#             '43.252.73.101:53557', '118.172.51.110:36552', '42.115.88.71:62225', '200.52.141.162:53281', '51.158.165.18:8811',
#             '89.189.172.88:8080', '181.143.79.154:48749', '182.52.238.119:40456', '162.244.83.192:3128', '47.88.16.118:3128',
#             '182.48.94.238:8080', '51.158.123.250:8811', '103.12.161.35:47091', '103.9.188.151:38439', '186.226.171.94:45514',
#             '36.92.79.41:55443', '163.172.189.32:8811', '36.89.183.241:47798', '51.158.108.135:8811', '193.233.183.145:8080',
#             '117.2.165.159:53281', '186.29.163.97:49787', '190.12.95.170:31260', '193.242.177.105:53281', '109.167.134.253:30710',
#             '188.165.16.230:3129', '45.236.171.82:999', '103.119.54.188:8888', '162.244.83.134:3128', '95.154.104.147:34153',
#            '87.197.156.62:23500', '202.179.7.182:56506', '178.134.71.138:35942', '109.232.106.236:35423', '201.75.31.88:53281',
#            '145.239.121.218:3129', '183.182.101.32:56814', '45.123.26.146:53281', '51.255.103.170:3129', '191.97.35.87:23500',
#             '192.140.42.28:39653', '160.119.131.114:48762', '144.217.101.242:3129', '103.240.206.152:55740', '163.172.136.226:8811',
#             '45.236.88.42:8880', '200.6.169.122:8888', '1.20.103.196:42792', '139.99.102.114:80', '114.199.111.162:80',
#            '124.41.243.22:47894', '103.204.171.202:53281', '50.192.195.69:52018', '186.125.59.8:44363', '103.28.121.58:3128',
#             '191.100.24.251:21776', '171.100.9.126:49163']
ip_addresses = ['15.165.85.203:3128', '46.254.217.54:53281', '163.172.136.226:8811', '41.76.117.194:30878',
                '194.228.129.189:61866', '85.198.133.19:43157', '170.247.152.132:43747', '176.110.121.90:21776',
                 '51.158.68.68:8811', '193.169.5.14:39602', '186.226.183.170:34497', '118.174.46.145:45330',
                 '94.103.12.44:3838', '35.198.246.77:808', '51.158.68.26:8811', '78.188.65.105:49743',
                 '180.180.175.70:59692', '94.103.12.42:3838', '139.180.185.83:3128', '139.180.188.35:3128',
                 '45.32.119.166:3128', '163.172.180.18:8811', '1.20.99.89:32963', '139.255.42.156:8888',
                '149.28.137.222:3128', '128.199.151.21:8080', '212.42.113.240:49297']

list_ip_aux = [] 

def do_requests(payload,url,contador):
    
    global ip_addresses
    global list_ip_aux
    
    headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'es-PE',
    'origin': 'https://web.cornershopapp.com',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
    'x-cornershop-version': 'platform=web, app=customers, release=2a014d2e2e1f021134ebf06eaf28be8f12894bdf' }

    hdrs = Headers(headers=True).generate()
    hdrs['Accept-Language'] = 'es-PE;q=0.8,es;q=0.7'
    
    
    rest = contador%100
    
    if rest == 0:
        np.random.shuffle(ip_addresses)
    #proxy_chosen = np.random.choice(ip_addresses)
    
    code_status = 400
    #index_ips = 0
    json_res = {}
    while(code_status != 200):
        
        try:
           
            proxy_chosen = ip_addresses[0]
            proxies = {"http": 'http://'+proxy_chosen, "https": 'https://'+proxy_chosen}
        
            r = requests.get(url,params=payload, headers=hdrs, proxies=proxies, timeout=15)
            print(r.url)
            code_status = r.status_code
            if code_status != 200:
                list_ip_aux.append(ip_addresses.pop(0))
                print('Error do_requests ',r.status_code)
                json_res = {}
            else:
                json_res = r.json()
            #index_ips += 1
        except Exception as e:
            print(e)
            list_ip_aux.append(ip_addresses.pop(0))
            code_status = 400
        
        if len(ip_addresses) == 0:
            ip_addresses = list_ip_aux
    
    return json_res

In [58]:
# MAIN FUNCTION

#list_dis = get_provincias() # request
#list_dis = ['Barranco',  'Breña',  'Jesus Maria', 'La Molina', 'Lince', 'Magdalena Del Mar', 'Magdalena Vieja', 'Miraflores', 'San Borja',
#             'San Isidro', 'San Miguel', 'Santa Anita', 'Santiago De Surco', 'Surquillo']
list_dis = ['Surquillo']
for distrito in list_dis:
    cont = 1
    print('Distrito > ', distrito)
    
    payload_1 = {'locality':distrito,'country':'PE'}
    url_1 = 'https://cornershopapp.com/api/v2/stores'
    json_markets = do_requests(payload_1,url_1,cont) # request
    
    if json_markets == []:
        continue
    
    top_markets = copy.deepcopy(json_markets)[0] #dict
    markets = copy.deepcopy(json_markets)[1:] #list
    
    if markets == []:
        markets = copy.deepcopy(json_markets)
    
    # get top markets in list with id
    list_top_market = format_market_by_category(top_markets)['id'].tolist()
    df_markets_by_cat = process_markets_by_category(markets)
    #markets with label of top_market
    df_markets_featured = set_top_markets(df_markets_by_cat, list_top_market)
    
    #exportar 1 distrito
    name_dis = distrito.lower()
    name_folder = distrito.replace(" ","").lower()
    df_markets_featured.to_csv(name_folder+'/'+name_dis+'_stores.csv',index=False,header=True,encoding='utf-8')
    

    df_markets_featured['extra'] = df_markets_featured.apply(build_tuple_ex,axis=1)
    lista_aux = df_markets_featured['extra'].tolist() # (id_branch, id_store)
    
    #list_id_branch = df_markets_featured['id_branch'].tolist()
    num_stores = len(lista_aux)
    stores_done = 1
    for tp in lista_aux:
        #request
        url_2 = 'https://cornershopapp.com/api/v1/branches/'+str(tp[0])+'?with_suspended_slots'
        #json_branch = departments_by_branch(str(tp[0]),distrito)
        cont += 1
        json_branch = do_requests(payload_1,url_2,cont)
        
        
        df_subcat, cont = format_subcategories_by_deparment(json_branch,str(tp[0]),cont)
        rows_subcat = df_subcat.shape[0]
        idx_subcat = 0
        #new_col =   # can be a list, a Series, an array or a scalar   
        df_subcat.insert(loc=idx_subcat, column='id_branch', value=str(tp[0]))
        
        #df_subcat['id_branch'] = str(tp[0])
        
        #exportar 2 por branch
        name_file_2 = name_dis+'_'+str(tp[1])+'_'+str(tp[0])
        df_subcat.to_csv(name_folder+'/'+name_file_2+'.csv',index=False,header=True,encoding='utf-8')
        
        lista_top_by_subcat = []
        if df_subcat.columns.tolist():
            lista_top_by_subcat = df_subcat['top_products'].tolist()
        
        unique_list_top_product = []
        for ls in lista_top_by_subcat:
            unique_list_top_product.extend(ls)
        
        # add extra
        df_subcat['extra'] = df_subcat.apply(build_tuple_aux,axis=1)
        lista_aux_subcat = df_subcat['extra'].tolist() #(iddep, idsubcat)
        
        
        df_products_by_branch = pd.DataFrame([],columns=[])
        for ap in lista_aux_subcat:

            df_products = pd.DataFrame([],columns=[])
            if str(ap[1]):
            
                #request
                #list_products = products_by_aisle(str(tp[0]),ap[1])
                url_3 ='https://cornershopapp.com/api/v1/branches/'+str(tp[0])+'/aisles/'+str(ap[1])+'/products'
                cont += 1
                list_products = do_requests({},url_3,cont)
                df_products = format_products(list_products)
                df_products = set_top_products(df_products,unique_list_top_product)
            else:
                data_default = [{'id_product':'','id_brand':'','name_brand':'','buy_unit':'','currency':'','unit_conversion_rate':'','purchasable':'','ppu_unit':'','ppu_amount':'','price':'','availability_status':'','nutritional_info':'','name':'','description':'','package':'','img_url':'','top_product':''}]
                df_products = pd.DataFrame(data_default)
            
            print(str(tp[0]),ap[1],cont) # id_branch, id_subcat, contador de requests
            #rows_products = df_products.shape[0]
            #col_idbranch = str(tp[0])*rows_products  # can be a list, a Series, an array or a scalar   
            #col_iddepa = str(ap[0])*rows_products
            #col_idsubcat = str(ap[1])*rows_products
            df_products.insert(loc=0, column='id_branch', value=str(tp[0]))
            df_products.insert(loc=1, column='id_depa', value=str(ap[0]))
            df_products.insert(loc=2, column='id_subcat', value=str(ap[1]))
            #df_products['id_branch'] = str(tp[0])
            #df_products['id_depa'] = str(ap[0])
            #df_products['id_subcat'] = str(ap[1])
            df_products_by_branch = df_products_by_branch.append(df_products)

            
        #exportar 3 all products
        name_file_3 = name_file_2+'_products'
        df_products_by_branch.to_csv(name_folder+'/'+name_file_3+'.csv',index=False,header=True,encoding='utf-8')
        
        print('Done '+ name_folder + ' > ' + str(stores_done) + ' / ' + str(num_stores) + ' stores')
        stores_done += 1

Distrito >  Surquillo
https://cornershopapp.com/api/v2/stores?locality=Surquillo&country=PE
https://cornershopapp.com/api/v1/branches/3804?with_suspended_slots&locality=Surquillo&country=PE
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3804/departments/C_45 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B8D90C88>, 'Connection to 15.165.85.203 timed out. (connect timeout=15)'))
https://cornershopapp.com/api/v1/branches/3804/departments/C_45
https://cornershopapp.com/api/v1/branches/3804/departments/C_virtual:others
https://cornershopapp.com/api/v1/branches/3804/aisles/C_327/products
3804 C_327 5
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3804/aisles/C_202/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B7D4A188>, 'Connection to 46.254.217.54 timed out. (connect timeout=15)'))
HTTPS

HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/4043/aisles/C_982/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B92D6F08>, 'Connection to 139.180.188.35 timed out. (connect timeout=15)'))
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/4043/aisles/C_982/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B791EE08>, 'Connection to 45.32.119.166 timed out. (connect timeout=15)'))
https://cornershopapp.com/api/v1/branches/4043/aisles/C_982/products
4043 C_982 38
https://cornershopapp.com/api/v1/branches/4043/aisles/C_572/products
4043 C_572 39
https://cornershopapp.com/api/v1/branches/4043/aisles/C_3/products
4043 C_3 40
Done surquillo > 5 / 27 stores
https://cornershopapp.com/api/v1/branches/3608?with_suspended_slots&locality=Surquillo&country=PE
https://cornershopapp.com/api/v1/bran

HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/5415/aisles/C_90/products (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out')))
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/5415/aisles/C_90/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B7A8D948>, 'Connection to 41.76.117.194 timed out. (connect timeout=15)'))
https://cornershopapp.com/api/v1/branches/5415/aisles/C_90/products
5415 C_90 107
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/5415/aisles/C_87/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B8DAA688>, 'Connection to 194.228.129.189 timed out. (connect timeout=15)'))
https://cornershopapp.com/api/v1/branches/5415/aisles/C_87/products
5415 C_87 108
https://cornershopapp.com/api/v1/branches/54

https://cornershopapp.com/api/v1/branches/4753/departments/C_63
https://cornershopapp.com/api/v1/branches/4753/departments/C_995
https://cornershopapp.com/api/v1/branches/4753/departments/C_962
https://cornershopapp.com/api/v1/branches/4753/departments/C_40
https://cornershopapp.com/api/v1/branches/4753/departments/C_virtual:others
https://cornershopapp.com/api/v1/branches/4753/aisles/C_34/products
4753 C_34 130
https://cornershopapp.com/api/v1/branches/4753/aisles/C_33/products
4753 C_33 131
https://cornershopapp.com/api/v1/branches/4753/aisles/C_32/products
4753 C_32 132
https://cornershopapp.com/api/v1/branches/4753/aisles/C_76/products
4753 C_76 133
https://cornershopapp.com/api/v1/branches/4753/aisles/C_77/products
4753 C_77 134
https://cornershopapp.com/api/v1/branches/4753/aisles/C_90/products
4753 C_90 135
https://cornershopapp.com/api/v1/branches/4753/aisles/C_87/products
4753 C_87 136
https://cornershopapp.com/api/v1/branches/4753/aisles/C_161/products
4753 C_161 137
https://

https://cornershopapp.com/api/v1/branches/4531/aisles/C_910/products
4531 C_910 209
https://cornershopapp.com/api/v1/branches/4531/aisles/C_97/products
4531 C_97 210
https://cornershopapp.com/api/v1/branches/4531/aisles/C_139/products
4531 C_139 211
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/4531/aisles/C_183/products (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out')))
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/4531/aisles/C_183/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B7817C48>, 'Connection to 15.165.85.203 timed out. (connect timeout=15)'))
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/4531/aisles/C_183/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B918F108>, 'Connection to 176.11

...

https://cornershopapp.com/api/v1/branches/7899/departments/C_512
https://cornershopapp.com/api/v1/branches/7899/departments/C_518
https://cornershopapp.com/api/v1/branches/7899/departments/C_962
https://cornershopapp.com/api/v1/branches/7899/departments/C_514
https://cornershopapp.com/api/v1/branches/7899/departments/C_511
https://cornershopapp.com/api/v1/branches/7899/departments/C_560
https://cornershopapp.com/api/v1/branches/7899/departments/C_530
https://cornershopapp.com/api/v1/branches/7899/departments/C_521
https://cornershopapp.com/api/v1/branches/7899/departments/C_529
https://cornershopapp.com/api/v1/branches/7899/departments/C_46
https://cornershopapp.com/api/v1/branches/7899/departments/C_515
https://cornershopapp.com/api/v1/branches/7899/departments/C_519
https://cornershopapp.com/api/v1/branches/7899/departments/C_523
https://cornershopapp.com/api/v1/branches/7899/departments/C_517
https://cornershopapp.com/api/v1/branches/7899/departments/C_528
https://cornershopapp.com/

HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/7899/aisles/C_147/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B91ED308>, 'Connection to 193.169.5.14 timed out. (connect timeout=15)'))
https://cornershopapp.com/api/v1/branches/7899/aisles/C_147/products
7899 C_147 746
https://cornershopapp.com/api/v1/branches/7899/aisles/C_148/products
7899 C_148 747
https://cornershopapp.com/api/v1/branches/7899/aisles/C_83/products
7899 C_83 748
https://cornershopapp.com/api/v1/branches/7899/aisles/C_85/products
7899 C_85 749
https://cornershopapp.com/api/v1/branches/7899/aisles/C_84/products
7899 C_84 750
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/7899/aisles/C_82/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B91E7888>, 'Connection to 15.165.85.203 timed out. (connect timeout=15)'))


HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/7899/aisles/C_372/products (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)))
https://cornershopapp.com/api/v1/branches/7899/aisles/C_372/products
7899 C_372 772
https://cornershopapp.com/api/v1/branches/7899/aisles/C_126/products
7899 C_126 773
https://cornershopapp.com/api/v1/branches/7899/aisles/C_992/products
7899 C_992 774
https://cornershopapp.com/api/v1/branches/7899/aisles/C_990/products
7899 C_990 775
https://cornershopapp.com/api/v1/branches/7899/aisles/C_249/products
7899 C_249 776
https://cornershopapp.com/api/v1/branches/7899/aisles/C_928/products
7899 C_928 777
https://cornershopapp.com/api/v1/branches/7899/aisles/C_360/products
7899 C_360 778
https://cornershopapp.com/api/v1/branches/7899/aisles/C_359/products
7899 C_359 779
https://cornershopapp.com/api/v1

HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/2991/departments/C_648 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B91E9508>, 'Connection to 51.158.68.68 timed out. (connect timeout=15)'))
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/2991/departments/C_648 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B9103048>, 'Connection to 186.226.183.170 timed out. (connect timeout=15)'))
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/2991/departments/C_648 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B7AD91C8>, 'Connection to 35.198.246.77 timed out. (connect timeout=15)'))
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/2991/departments/C_648 (

https://cornershopapp.com/api/v1/branches/3058/departments/C_526
https://cornershopapp.com/api/v1/branches/3058/departments/C_995
https://cornershopapp.com/api/v1/branches/3058/departments/C_514
https://cornershopapp.com/api/v1/branches/3058/aisles/C_362/products
3058 C_362 888
https://cornershopapp.com/api/v1/branches/3058/aisles/C_365/products
3058 C_365 889
https://cornershopapp.com/api/v1/branches/3058/aisles/C_360/products
3058 C_360 890
https://cornershopapp.com/api/v1/branches/3058/aisles/C_359/products
3058 C_359 891
https://cornershopapp.com/api/v1/branches/3058/aisles/C_585/products
3058 C_585 892
https://cornershopapp.com/api/v1/branches/3058/aisles/C_357/products
3058 C_357 893
https://cornershopapp.com/api/v1/branches/3058/aisles/C_973/products
3058 C_973 894
https://cornershopapp.com/api/v1/branches/3058/aisles/C_972/products
3058 C_972 895
https://cornershopapp.com/api/v1/branches/3058/aisles/C_974/products
3058 C_974 896
https://cornershopapp.com/api/v1/branches/3058/ai

In [103]:
df1=pd.DataFrame([{'as':}],columns=['d'])
#df1 = df1.append(pd.DataFrame([]))
if df1.columns.tolist():
    print("df")

,d


In [96]:
hdrs = Headers(headers=True).generate()
hdrs['Accept-Language'] = 'es-PE;q=0.8,es;q=0.7'
hdrs

{'Accept': '*/*',
 'Connection': 'keep-alive',
 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:52.5.0) Gecko/20100101 Firefox/52.5.0',
 'DNT': '1',
 'Upgrade-Insecure-Requests': '1',
 'Referer': 'https://google.com',
 'Pragma': 'no-cache',
 'Accept-Language': 'es-PE;q=0.8,es;q=0.7'}

In [33]:
def test_requests(payload,url,proxy):

    hdrs = Headers(headers=True).generate()
    hdrs['Accept-Language'] = 'es-PE;q=0.8,es;q=0.7'

    proxies = {"http": 'http://'+proxy, "https": 'https://'+proxy}

    response = False
    
    try:
        print("Proxy > ", proxy)
        r = requests.get(url,params=payload, headers=hdrs, proxies=proxies, timeout=15)
        print(r.url)
        code_status = r.status_code
        if code_status != 200:
            print('Error do_requests ',r.status_code)
        else:
            response = True
    except Exception as e:
        print(e) 
    
    return response

ip_addresses = ['1.32.59.217:46162','197.242.206.64:34576','93.91.112.247:41258','15.165.85.203:3128','195.138.73.54:47108',
 '119.82.253.175:31500','197.255.253.210:55263','46.254.217.54:53281','163.172.136.226:8811','41.76.117.194:30878',
 '128.0.179.234:41258','194.228.129.189:61866','182.176.228.147:57472','103.75.162.68:36015','176.98.76.203:36651',
 '185.138.123.78:55337','104.148.76.142:3128','85.198.133.19:43157','170.247.152.132:43747','176.110.121.90:21776',
 '165.16.3.54:53281','80.106.247.145:53410','194.58.182.73:41258','51.158.68.68:8811','202.138.241.166:8080',
 '193.169.5.14:39602','195.4.164.127:8080','109.87.40.23:44343','194.8.146.167:50510','109.75.140.158:59916','158.58.133.106:41258',
 '125.26.7.124:61642','177.126.216.149:46320','186.226.183.170:34497','118.174.46.145:45330','185.128.37.4:8080',
 '103.28.121.58:3128','110.4.47.140:3128','61.19.40.50:33665','62.182.114.164:53309','94.103.12.44:3838',
 '203.153.20.64:53281','85.10.219.100:1080','95.80.252.189:52371','35.198.246.77:808','51.158.68.26:8811',
 '78.188.65.105:49743','89.36.195.238:39131','101.51.141.46:37858','213.6.38.50:45540','46.151.108.6:30874',
 '180.180.175.70:59692','88.198.33.232:1080','194.44.225.34:53281','181.102.197.38:7071','181.117.176.236:61358',
 '51.158.165.18:8811','94.103.12.42:3838','168.90.50.118:58192','78.8.45.68:8080','125.212.218.43:8888',
 '181.102.192.139:7071','103.220.207.17:42670','93.117.72.27:32468','47.88.16.118:3128','104.148.76.167:3128',
 '139.180.185.83:3128','139.180.188.35:3128','45.32.119.166:3128','103.133.204.169:8080','104.148.76.136:3128',
 '190.84.232.87:47295','163.172.180.18:8811','104.148.76.141:3128','41.75.123.49:41263','1.20.99.89:32963','139.255.42.156:8888',
 '149.28.137.222:3128','128.199.151.21:8080','212.42.113.240:49297']


id_br = '3574'
id_ais = 'C_444'
url_aux = 'https://cornershopapp.com/api/v1/branches/'+id_br+'/aisles/'+id_ais+'/products'

good_ip = []
bad_ip = []
for ip in ip_addresses:
    response = test_requests({},url_aux,ip)
    if response:
        good_ip.append(ip)
    else:
        bad_ip.append(ip)




Proxy >  1.32.59.217:46162
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3574/aisles/C_444/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B7803748>, 'Connection to 1.32.59.217 timed out. (connect timeout=15)'))
Proxy >  197.242.206.64:34576
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3574/aisles/C_444/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B7802708>, 'Connection to 197.242.206.64 timed out. (connect timeout=15)'))
Proxy >  93.91.112.247:41258
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3574/aisles/C_444/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B8DDC308>, 'Connection to 93.91.112.247 timed out. (connect timeout=15)'))
Proxy >  15.165.85.203:3128
https

https://cornershopapp.com/api/v1/branches/3574/aisles/C_444/products
Proxy >  118.174.46.145:45330
https://cornershopapp.com/api/v1/branches/3574/aisles/C_444/products
Proxy >  185.128.37.4:8080
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3574/aisles/C_444/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000285B8D88208>, 'Connection to 185.128.37.4 timed out. (connect timeout=15)'))
Proxy >  103.28.121.58:3128
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3574/aisles/C_444/products (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1076)')))
Proxy >  110.4.47.140:3128
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3574/aisles/C_444/products (Caused by ConnectTim

HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3574/aisles/C_444/products (Caused by ProxyError('Cannot connect to proxy.', timeout('_ssl.c:1059: The handshake operation timed out')))
Proxy >  104.148.76.167:3128
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3574/aisles/C_444/products (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out')))
Proxy >  139.180.185.83:3128
https://cornershopapp.com/api/v1/branches/3574/aisles/C_444/products
Proxy >  139.180.188.35:3128
https://cornershopapp.com/api/v1/branches/3574/aisles/C_444/products
Proxy >  45.32.119.166:3128
https://cornershopapp.com/api/v1/branches/3574/aisles/C_444/products
Proxy >  103.133.204.169:8080
HTTPSConnectionPool(host='cornershopapp.com', port=443): Max retries exceeded with url: /api/v1/branches/3574/aisles/C_444/products (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection ob

In [34]:
good_ip

['15.165.85.203:3128',
 '46.254.217.54:53281',
 '163.172.136.226:8811',
 '41.76.117.194:30878',
 '194.228.129.189:61866',
 '85.198.133.19:43157',
 '170.247.152.132:43747',
 '176.110.121.90:21776',
 '51.158.68.68:8811',
 '193.169.5.14:39602',
 '186.226.183.170:34497',
 '118.174.46.145:45330',
 '94.103.12.44:3838',
 '35.198.246.77:808',
 '51.158.68.26:8811',
 '78.188.65.105:49743',
 '180.180.175.70:59692',
 '94.103.12.42:3838',
 '139.180.185.83:3128',
 '139.180.188.35:3128',
 '45.32.119.166:3128',
 '163.172.180.18:8811',
 '1.20.99.89:32963',
 '139.255.42.156:8888',
 '149.28.137.222:3128',
 '128.199.151.21:8080',
 '212.42.113.240:49297']

In [127]:
bad_ip

['158.58.133.106:41258',
 '103.105.77.48:8181',
 '202.150.143.59:54397',
 '197.53.177.124:8080',
 '103.12.160.200:53281',
 '196.44.67.172:53281',
 '134.249.141.148:80',
 '200.52.141.162:53281',
 '207.157.220.8:8080',
 '156.155.155.11:40715',
 '51.158.68.133:8811',
 '51.158.172.165:8811',
 '193.33.241.61:45837',
 '94.228.21.66:30784',
 '75.150.251.146:3128',
 '95.181.37.114:45517',
 '88.255.102.99:54023',
 '195.62.70.254:8080',
 '178.134.155.82:48146',
 '119.82.249.4:46398',
 '190.145.148.34:56368',
 '1.10.188.42:48721',
 '118.175.93.171:32866',
 '202.169.38.198:30612',
 '93.157.163.66:35081',
 '179.17.254.224:8080',
 '194.114.128.149:61213',
 '103.86.187.242:23500',
 '91.225.226.39:53088',
 '103.100.83.238:36598',
 '154.117.143.126:8080',
 '85.159.48.170:40014',
 '104.171.124.73:8080',
 '179.205.15.101:8080',
 '124.41.213.211:41828',
 '46.90.83.46:8080',
 '201.180.176.169:8080',
 '78.46.40.154:8118',
 '181.196.247.162:37107',
 '102.164.199.76:47149',
 '94.141.244.39:34166',
 '87.126.43

In [201]:
# 2902 - PlazaVea
# gather branch data and departments id
def products_by_aisle(id_branch, id_aisle):
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'es-PE;q=0.8,es;q=0.7,en-US;q=0.6,en;q=0.5,es-US;q=0.4,ko-KR;q=0.3,ko;q=0.2',
    'Cache-Control': 'no-cache'}
    #https://cornershopapp.com/api/v1/branches/2896/aisles/C_134/products
    payload = {}
    hdrs = Headers(os="mac", headers=True).generate()
    hdrs['Accept-Language'] = 'es-PE;q=0.8,es;q=0.7'
    r = requests.get('https://cornershopapp.com/api/v1/branches/'+id_branch+'/aisles/'+id_aisle+'/products',headers=hdrs)
    r.encoding = 'utf-8'
    
    print(r)
    print(r.url)
    print(r.text)
    print(r.decode().json())
    json_res = r.json()
    
    return json_res
products_by_aisle('3574', 'C_444')
#https://cornershopapp.com/api/v1/branches/4105/aisles/C_325/products

<Response [200]>
https://cornershopapp.com/api/v1/branches/3574/aisles/C_444/products
[{"id":1118718,"brand":{"id":28944,"name":"B. Toys"},"buy_unit":"UN","default_buy_unit":"UN","variable_weight":null,"currency":"PEN","unit_conversion_rate":null,"purchasable":true,"pricing":{"original_price":null,"price":{"label":"Precio","amount":62.0,"currency":"PEN"},"price_per_unit":null},"price_per_unit":"","price":62.0,"original_price":null,"label":null,"description_sections":[],"availability_status":null,"nutritional_info":null,"name":"Bloques suaves pequeños","description":"Edifíquelos o apriételos Las imágenes esculpidas a mano adornan cada lado de cada bloque es:\nApilable\nCompresible\nFlotable\nChirriante\nMasticable (sin BPA)","package":null,"img_url":"https://s.cornershopapp.com/product-images/1118718.jpg?versionId=p53fCeWCKgzvsgpglWxXUoW1t1Fvot8e"},{"id":1118719,"brand":{"id":28944,"name":"B. Toys"},"buy_unit":"UN","default_buy_unit":"UN","variable_weight":null,"currency":"PEN","unit_co

AttributeError: 'Response' object has no attribute 'decode'